In [ ]:
import pandas as pd
import requests

In [ ]:
category_data_df = pd.read_csv('/Users/stevie/repos/lingo_kit_data/dataframes/category_data.csv')
foundational_words_df = pd.read_csv('/Users/stevie/repos/lingo_kit_data/dataframes/foundational_words.csv')
spotify_lessons_df = pd.read_csv('/Users/stevie/repos/lingo_kit_data/dataframes/spotify_lessons.csv')

In [ ]:
foundational_words_df[foundational_words_df['part_of_speech'] == 'verb']

In [ ]:
df = pd.concat([category_data_df, foundational_words_df, spotify_lessons_df], ignore_index=True)

In [ ]:
len(df), df.columns

In [ ]:
import uuid
def get_term_hash(english_text, italian_text):
    hash = str(uuid.uuid5(uuid.NAMESPACE_DNS, f"{english_text}-{italian_text}"))
    return hash
print(get_term_hash('person', 'persone'))

In [ ]:
df.drop_duplicates(subset=['english_term', 'italian_term'], inplace=True)
len(df)

In [ ]:
temp = df.copy()
# temp = temp[temp['part_of_speech']=='phrase']
print(len(temp))

In [ ]:
from tqdm import tqdm
pos = set()
count = 0
for i, row in tqdm(temp.iterrows(), total=len(temp)):
    term_hash = get_term_hash(row['english_term'], row['italian_term'])
    response = requests.get(f'http://127.0.0.1:8000/term/{term_hash}')
    assert(response.status_code == 200)
    term_dict = response.json()
    if term_dict['english_base_term'] != row['english_base_term'] or term_dict['italian_base_term'] != row['italian_base_term']:
        pos.add(row['part_of_speech'])
        count += 1
        if row['part_of_speech'] != 'phrase':
            print(f"TERM_DICT: {term_dict}")
            print(f"ROW: {row.to_dict()}")
        # term_dict['english_base_term'] = row['english_base_term']
        # term_dict['italian_base_term'] = row['italian_base_term']
        # call put service to update
        # response = requests.put(f"http://127.0.0.1:8000/update-term/{term_hash}", json=term_dict)
        assert(response.status_code == 200)
print(pos, count)

In [ ]:
len(requests.get(f'http://127.0.0.1:8000/terms').json())